In [8]:
import matplotlib.pyplot as plt
import numpy as np
from termcolor import colored

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy, Mean
from tensorflow.keras.optimizers import SGD

In [22]:
''' ready for dataset'''
n_train, n_validation, n_test = 1000, 300, 300

train_x = np.random.normal(0, 1, size=(n_train, 1)).astype('float32')
train_x_noise = train_x + 0.2 * np.random.normal(0, 1, size=(n_train, 1))
train_y = (train_x_noise > 0).astype(np.int32)

validation_x = np.random.normal(0, 1, size=(n_train, 1)).astype('float32')
validation_x_noise = validation_x + 0.2 * np.random.normal(0, 1, size=(n_train, 1))
validation_y = (validation_x_noise > 0).astype(np.int32)

test_x = np.random.normal(0, 1, size=(n_train, 1)).astype('float32')
test_x_noise = test_x + 0.2 * np.random.normal(0, 1, size=(n_train, 1))
test_y = (test_x_noise > 0).astype(np.int32)


''' split dataset'''
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_ds = train_ds.shuffle(n_train).batch(8)

validation_ds = tf.data.Dataset.from_tensor_slices((validation_x, validation_y))
validation_ds = validation_ds.batch(n_validation)

test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_ds = test_ds.batch(n_test)


''' parameter ready '''
train_loss = Mean()
train_acc = SparseCategoricalAccuracy()

validation_loss = Mean()
validation_acc = SparseCategoricalAccuracy()

test_loss = Mean()
test_acc = SparseCategoricalAccuracy()


''' modeling '''
model = Sequential()
model.add(Dense(units=2, activation='softmax'))

loss_object = SparseCategoricalCrossentropy()
optimizer = SGD(learning_rate=1)

train_losses, validation_losses = [], []
train_accs, validation_accs = [], []

EPOCHS = 10

@tf.function
def train_step(x, y):
    global model, loss_object
    global train_loss, train_acc
    
    with tf.GradientTape() as tape:
        predictions = model(x)
        loss = loss_object(y, predictions)
        
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
    train_loss(loss)
    train_acc(y, predictions)

@tf.function
def validation_step():
    global validation_ds, model, loss_object
    global train_loss, train_acc
    
    for x, y in validation_ds:
        predictions = model(x)
        loss = loss_object(y, predictions)
        
        validation_loss(loss)
        validation_acc(y, predictions)

# @tf.function        
# def train_reporter():
#     global epoch, train_loss, train_acc
#     global valdiation_loss, validation_acc
    
#     print(colored('Epochs: ', 'red', 'on_white'), epoch + 1)
#     template1 = f'Train Loss : {train_loss.result():.4f}\t Train Accuracy: {train_acc.result()*100:.2f}%\n' 
#     template2 = f'Validation Loss : {validation_loss.result():.4f}\t Validation Accuracy: {validation_acc.result()*100:.2f}%\n'
    
#     print(template1, template2)
    # print(template.format(train_loss.result(), train_acc.result()*100, validation_loss.result(), validation_acc.result()*100))

@tf.function
def metric_resetter():
    global train_losses, train_accs, train_loss, train_acc
    global validation_losses, validation_accs, validation_loss, validation_acc
    
    train_losses.append(train_loss.result())
    train_accs.append(train_acc.result())
    validation_losses.append(validation_loss.result())
    validation_accs.append(validation_acc.result())
    
    train_loss.reset_states()
    train_acc.reset_states()
    validation_loss.reset_states()
    validation_acc.reset_states()

def final_result_visualization():
    global train_losses, validation_losses
    global train_acss, validation_accs
    
    fig, axes = plt.subplots(figsize=(12, 8))
    axes[0].plot(train_losses, label= 'Train Loss')
    axes[0].plot(validation_losses, label='Validation Loss')
    axes[1].plot(train_accs, label='Train Acc')
    axes[1].plot(validation_losses, label='Valdiation Acc')
    axes[0].tick_params(labelsize=13)
    axes[1].tick_params(labelsize=13)
    
    axes[0].set_ylabel('Binary Cross Entropy', fontsize=13)
    axes[1].set_ylabel('Accuracy', fontsize=13)
    axes[1].set_xlabel('Epoch', fontsize=13)
    

for epoch in range(EPOCHS):
    for x, y in train_ds:
        train_step(x, y)
        
    validation_step()
    
    print(colored('Epochs: ', 'red', 'on_white'), epoch + 1)
    template1 = f'Train Loss : {train_loss.result():.4f}\t Train Accuracy: {train_acc.result()*100:.2f}%\n' 
    template2 = f'Validation Loss : {validation_loss.result():.4f}\t Validation Accuracy: {validation_acc.result()*100:.2f}%\n'
    
    print(template1, template2)
    
    metric_resetter()

''' test set '''
for x, y in test_ds:
    predictions = model(x)
    loss = loss_object(y, predictions)

    test_loss(loss)
    test_acc(y, predictions)

print(colored('Epochs: ', 'cyan', 'on_white'), epoch + 1)
template = 'test Loss : {:.4f}\t test Accuracy: {:.2f}%\n'

print(template.format(test_loss.result(), test_acc.result()*100))

Epochs:  1
Train Loss : 0.2127	 Train Accuracy: 92.20%
 Validation Loss : 0.1512	 Validation Accuracy: 93.20%

Epochs:  2
Train Loss : 0.1690	 Train Accuracy: 93.10%
 Validation Loss : 0.1461	 Validation Accuracy: 93.20%

Epochs:  3
Train Loss : 0.1678	 Train Accuracy: 92.00%
 Validation Loss : 0.1449	 Validation Accuracy: 93.40%

Epochs:  4
Train Loss : 0.1662	 Train Accuracy: 92.60%
 Validation Loss : 0.1419	 Validation Accuracy: 92.90%

Epochs:  5
Train Loss : 0.1639	 Train Accuracy: 92.70%
 Validation Loss : 0.1782	 Validation Accuracy: 92.30%

Epochs:  6
Train Loss : 0.1661	 Train Accuracy: 92.60%
 Validation Loss : 0.1435	 Validation Accuracy: 93.20%

Epochs:  7
Train Loss : 0.1654	 Train Accuracy: 92.80%
 Validation Loss : 0.1411	 Validation Accuracy: 93.00%

Epochs:  8
Train Loss : 0.1678	 Train Accuracy: 92.30%
 Validation Loss : 0.1437	 Validation Accuracy: 93.40%

Epochs:  9
Train Loss : 0.1671	 Train Accuracy: 92.90%
 Validation Loss : 0.1476	 Validation Accuracy: 93.50%

E